In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving all.anno to all.anno
Saving all.code to all.code
User uploaded file "all.anno" with length 1382085 bytes
User uploaded file "all.code" with length 906732 bytes


In [ ]:
import collections
import os
import pathlib
import re
import string
import sys
import tempfile
import time
import json

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
!pip install -q tensorflow-text-nightly
!pip install -q tensorflow_datasets
!pip install -q tf-nightly

!pip install tensorflow
import tensorflow_text as text
import tensorflow as tf

     |████████████████████████████████| 4.4MB 5.4MB/s 
     |████████████████████████████████| 454.3MB 23kB/s 
     |████████████████████████████████| 1.2MB 28.6MB/s 
     |████████████████████████████████| 4.0MB 26.9MB/s 
     |████████████████████████████████| 471kB 52.2MB/s 
     |████████████████████████████████| 6.0MB 43.2MB/s 
     |████████████████████████████████| 4.0MB 38.4MB/s 
     |████████████████████████████████| 3.9MB 34.3MB/s 
ERROR: tensorflow 2.4.1 has requirement gast==0.3.3, but you'll have gast 0.4.0 which is incompatible.
ERROR: tensorflow 2.4.1 has requirement grpcio~=1.32.0, but you'll have grpcio 1.34.1 which is incompatible.
ERROR: tensorflow 2.4.1 has requirement h5py~=2.10.0, but you'll have h5py 3.1.0 which is incompatible.
     |████████████████████████████████| 2.9MB 4.6MB/s 
     |████████████████████████████████| 3.8MB 16.7MB/s 
ERROR: tf-nightly 2.6.0.dev20210329 has requirement gast==0.4.0, but you'll have gast 0.3.3 which is incompatible.
ERROR: tf-n

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
tf.keras.utils.get_file(
    f"en_py_converter.zip",
    f"https://storage.googleapis.com/download.tensorflow.org/models/en_py_converter.zip",
    cache_dir='.', cache_subdir='', extract=True
)

'./en_py_converter.zip'

In [ ]:
model_name="en_py_converter"

In [ ]:
tokenizers = tf.saved_model.load(model_name)

In [ ]:
def get_angles(pos, i, d_model):
  angle_rates = 1 / np.power(10000, (2 * (i//2)) / np.float32(d_model))
  return pos * angle_rates

In [ ]:
def positional_encoding(position, d_model):
  angle_rads = get_angles(np.arange(position)[:, np.newaxis],
                          np.arange(d_model)[np.newaxis, :],
                          d_model)
  
  # apply sin to even indices in the array; 2i
  angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])
  
  # apply cos to odd indices in the array; 2i+1
  angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])
    
  pos_encoding = angle_rads[np.newaxis, ...]
    
  return tf.cast(pos_encoding, dtype=tf.float32)

In [ ]:
def create_padding_mask(seq):
  seq = tf.cast(tf.math.equal(seq, 0), tf.float32)
  
  # add extra dimensions to add the padding
  # to the attention logits.
  return seq[:, tf.newaxis, tf.newaxis, :]  # (batch_size, 1, 1, seq_len)

In [ ]:
def create_look_ahead_mask(size):
  mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)
  return mask  # (seq_len, seq_len)

In [ ]:
def scaled_dot_product_attention(q, k, v, mask):
  """Calculate the attention weights.
  q, k, v must have matching leading dimensions.
  k, v must have matching penultimate dimension, i.e.: seq_len_k = seq_len_v.
  The mask has different shapes depending on its type(padding or look ahead) 
  but it must be broadcastable for addition.
  
  Args:
    q: query shape == (..., seq_len_q, depth)
    k: key shape == (..., seq_len_k, depth)
    v: value shape == (..., seq_len_v, depth_v)
    mask: Float tensor with shape broadcastable 
          to (..., seq_len_q, seq_len_k). Defaults to None.
    
  Returns:
    output, attention_weights
  """

  matmul_qk = tf.matmul(q, k, transpose_b=True)  # (..., seq_len_q, seq_len_k)
  
  # scale matmul_qk
  dk = tf.cast(tf.shape(k)[-1], tf.float32)
  scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)

  # add the mask to the scaled tensor.
  if mask is not None:
    scaled_attention_logits += (mask * -1e9)  

  # softmax is normalized on the last axis (seq_len_k) so that the scores
  # add up to 1.
  attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)  # (..., seq_len_q, seq_len_k)

  output = tf.matmul(attention_weights, v)  # (..., seq_len_q, depth_v)

  return output, attention_weights

In [ ]:
def print_out(q, k, v):
  temp_out, temp_attn = scaled_dot_product_attention(
      q, k, v, None)
  print ('Attention weights are:')
  print (temp_attn)
  print ('Output is:')
  print (temp_out)

In [ ]:
class MultiHeadAttention(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads):
    super(MultiHeadAttention, self).__init__()
    self.num_heads = num_heads
    self.d_model = d_model
    
    assert d_model % self.num_heads == 0
    
    self.depth = d_model // self.num_heads
    
    self.wq = tf.keras.layers.Dense(d_model)
    self.wk = tf.keras.layers.Dense(d_model)
    self.wv = tf.keras.layers.Dense(d_model)
    
    self.dense = tf.keras.layers.Dense(d_model)
        
  def split_heads(self, x, batch_size):
    """Split the last dimension into (num_heads, depth).
    Transpose the result such that the shape is (batch_size, num_heads, seq_len, depth)
    """
    x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
    return tf.transpose(x, perm=[0, 2, 1, 3])
    
  def call(self, v, k, q, mask):
    batch_size = tf.shape(q)[0]
    
    q = self.wq(q)  # (batch_size, seq_len, d_model)
    k = self.wk(k)  # (batch_size, seq_len, d_model)
    v = self.wv(v)  # (batch_size, seq_len, d_model)
    
    q = self.split_heads(q, batch_size)  # (batch_size, num_heads, seq_len_q, depth)
    k = self.split_heads(k, batch_size)  # (batch_size, num_heads, seq_len_k, depth)
    v = self.split_heads(v, batch_size)  # (batch_size, num_heads, seq_len_v, depth)
    
    # scaled_attention.shape == (batch_size, num_heads, seq_len_q, depth)
    # attention_weights.shape == (batch_size, num_heads, seq_len_q, seq_len_k)
    scaled_attention, attention_weights = scaled_dot_product_attention(
        q, k, v, mask)
    
    scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])  # (batch_size, seq_len_q, num_heads, depth)

    concat_attention = tf.reshape(scaled_attention, 
                                  (batch_size, -1, self.d_model))  # (batch_size, seq_len_q, d_model)

    output = self.dense(concat_attention)  # (batch_size, seq_len_q, d_model)
        
    return output, attention_weights

In [ ]:
def point_wise_feed_forward_network(d_model, dff):
  return tf.keras.Sequential([
      tf.keras.layers.Dense(dff, activation='relu'),  # (batch_size, seq_len, dff)
      tf.keras.layers.Dense(d_model)  # (batch_size, seq_len, d_model)
  ])

In [ ]:
class EncoderLayer(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads, dff, rate=0.1):
    super(EncoderLayer, self).__init__()

    self.mha = MultiHeadAttention(d_model, num_heads)
    self.ffn = point_wise_feed_forward_network(d_model, dff)

    self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    
    self.dropout1 = tf.keras.layers.Dropout(rate)
    self.dropout2 = tf.keras.layers.Dropout(rate)
    
  def __call__(self, x, training, mask):

    attn_output, _ = self.mha(x, x, x, mask)  # (batch_size, input_seq_len, d_model)
    attn_output = self.dropout1(attn_output, training=training)
    out1 = self.layernorm1(x + attn_output)  # (batch_size, input_seq_len, d_model)
    
    ffn_output = self.ffn(out1)  # (batch_size, input_seq_len, d_model)
    ffn_output = self.dropout2(ffn_output, training=training)
    out2 = self.layernorm2(out1 + ffn_output)  # (batch_size, input_seq_len, d_model)
    
    return out2

In [ ]:
class DecoderLayer(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads, dff, rate=0.1):
    super(DecoderLayer, self).__init__()

    self.mha1 = MultiHeadAttention(d_model, num_heads)
    self.mha2 = MultiHeadAttention(d_model, num_heads)

    self.ffn = point_wise_feed_forward_network(d_model, dff)
 
    self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    self.layernorm3 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    
    self.dropout1 = tf.keras.layers.Dropout(rate)
    self.dropout2 = tf.keras.layers.Dropout(rate)
    self.dropout3 = tf.keras.layers.Dropout(rate)
    
    
  def call(self, x, enc_output, training, 
           look_ahead_mask, padding_mask):
    # enc_output.shape == (batch_size, input_seq_len, d_model)

    attn1, attn_weights_block1 = self.mha1(x, x, x, look_ahead_mask)  # (batch_size, target_seq_len, d_model)
    attn1 = self.dropout1(attn1, training=training)
    out1 = self.layernorm1(attn1 + x)
    
    attn2, attn_weights_block2 = self.mha2(
        enc_output, enc_output, out1, padding_mask)  # (batch_size, target_seq_len, d_model)
    attn2 = self.dropout2(attn2, training=training)
    out2 = self.layernorm2(attn2 + out1)  # (batch_size, target_seq_len, d_model)
    
    ffn_output = self.ffn(out2)  # (batch_size, target_seq_len, d_model)
    ffn_output = self.dropout3(ffn_output, training=training)
    out3 = self.layernorm3(ffn_output + out2)  # (batch_size, target_seq_len, d_model)
    
    return out3, attn_weights_block1, attn_weights_block2

In [ ]:
class Encoder(tf.keras.layers.Layer):
  def __init__(self, num_layers, d_model, num_heads, dff, input_vocab_size,
               maximum_position_encoding, rate=0.1):
    super(Encoder, self).__init__()

    self.d_model = d_model
    self.num_layers = num_layers
    
    self.embedding = tf.keras.layers.Embedding(input_vocab_size, d_model)
    self.pos_encoding = positional_encoding(maximum_position_encoding, 
                                            self.d_model)
    
    
    self.enc_layers = [EncoderLayer(d_model, num_heads, dff, rate) 
                       for _ in range(num_layers)]
  
    self.dropout = tf.keras.layers.Dropout(rate)
        
  def call(self, x, training, mask):

    seq_len = tf.shape(x)[1]
    
    # adding embedding and position encoding.
    x = self.embedding(x)  # (batch_size, input_seq_len, d_model)
    x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
    x += self.pos_encoding[:, :seq_len, :]

    x = self.dropout(x, training=training)
    
    for i in range(self.num_layers):
      x = self.enc_layers[i](x, training, mask)
    
    return x  # (batch_size, input_seq_len, d_model)

In [ ]:
class Decoder(tf.keras.layers.Layer):
  def __init__(self, num_layers, d_model, num_heads, dff, target_vocab_size,
               maximum_position_encoding, rate=0.1):
    super(Decoder, self).__init__()

    self.d_model = d_model
    self.num_layers = num_layers
    
    self.embedding = tf.keras.layers.Embedding(target_vocab_size, d_model)
    self.pos_encoding = positional_encoding(maximum_position_encoding, d_model)
    
    self.dec_layers = [DecoderLayer(d_model, num_heads, dff, rate) 
                       for _ in range(num_layers)]
    self.dropout = tf.keras.layers.Dropout(rate)
    
  def call(self, x, enc_output, training, 
           look_ahead_mask, padding_mask):

    seq_len = tf.shape(x)[1]
    attention_weights = {}
    
    x = self.embedding(x)  # (batch_size, target_seq_len, d_model)
    x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
    x += self.pos_encoding[:, :seq_len, :]
    
    x = self.dropout(x, training=training)

    for i in range(self.num_layers):
      x, block1, block2 = self.dec_layers[i](x, enc_output, training,
                                             look_ahead_mask, padding_mask)
      
      attention_weights['decoder_layer{}_block1'.format(i+1)] = block1
      attention_weights['decoder_layer{}_block2'.format(i+1)] = block2
    
    # x.shape == (batch_size, target_seq_len, d_model)
    return x, attention_weights

In [ ]:
class Transformer(tf.keras.Model):
  def __init__(self, num_layers, d_model, num_heads, dff, input_vocab_size, 
               target_vocab_size, pe_input, pe_target, rate=0.1):
    super(Transformer, self).__init__()

    self.tokenizer = Encoder(num_layers, d_model, num_heads, dff, 
                           input_vocab_size, pe_input, rate)

    self.decoder = Decoder(num_layers, d_model, num_heads, dff, 
                           target_vocab_size, pe_target, rate)

    self.final_layer = tf.keras.layers.Dense(target_vocab_size)
    
  def call(self, inp, tar, training, enc_padding_mask, 
           look_ahead_mask, dec_padding_mask):

    enc_output = self.tokenizer(inp, training, enc_padding_mask)  # (batch_size, inp_seq_len, d_model)
    
    # dec_output.shape == (batch_size, tar_seq_len, d_model)
    dec_output, attention_weights = self.decoder(
        tar, enc_output, training, look_ahead_mask, dec_padding_mask)
    
    final_output = self.final_layer(dec_output)  # (batch_size, tar_seq_len, target_vocab_size)
    
    return final_output, attention_weights

In [ ]:
num_layers = 4
d_model = 128
dff = 512
num_heads = 8
dropout_rate = 0.1

In [ ]:
transformer = Transformer(
    num_layers=num_layers,
    d_model=d_model,
    num_heads=num_heads,
    dff=dff,
    input_vocab_size=tokenizers.en.get_vocab_size(),
    target_vocab_size=tokenizers.py.get_vocab_size(), 
    pe_input=1000, 
    pe_target=1000,
    rate=dropout_rate)

In [ ]:
latest = tf.train.latest_checkpoint("/content/drive/MyDrive/checkpoints")
print(latest)
temp_input = tf.random.uniform((64, 38), dtype=tf.int64, minval=0, maxval=200)
temp_target = tf.random.uniform((64, 36), dtype=tf.int64, minval=0, maxval=200)

fn_out, _ = transformer(temp_input, temp_target, training=False, 
                               enc_padding_mask=None, 
                               look_ahead_mask=None,
                               dec_padding_mask=None)

fn_out.shape  # (batch_size, tar_seq_len, target_vocab_size)
transformer.load_weights(latest)

/content/drive/MyDrive/checkpoints/best_transformer


In [ ]:
def create_masks(inp, tar):
  # Encoder padding mask
  enc_padding_mask = create_padding_mask(inp)
  
  # Used in the 2nd attention block in the decoder.
  # This padding mask is used to mask the encoder outputs.
  dec_padding_mask = create_padding_mask(inp)
  
  # Used in the 1st attention block in the decoder.
  # It is used to pad and mask future tokens in the input received by 
  # the decoder.
  look_ahead_mask = create_look_ahead_mask(tf.shape(tar)[1])
  dec_target_padding_mask = create_padding_mask(tar)
  combined_mask = tf.maximum(dec_target_padding_mask, look_ahead_mask)
  
  return enc_padding_mask, combined_mask, dec_padding_mask

In [ ]:
def evaluate(sentence, max_length=60):
  # inp sentence is portuguese, hence adding the start and end token
  sentence = tf.convert_to_tensor([sentence])
  sentence = tokenizers.en.tokenize(sentence).to_tensor()

  encoder_input = sentence
  
  # as the target is english, the first word to the transformer should be the
  # english start token.
  start, end = tokenizers.py.tokenize([''])[0]
  output = tf.convert_to_tensor([start])
  output = tf.expand_dims(output, 0)
    
  for i in range(max_length):
    enc_padding_mask, combined_mask, dec_padding_mask = create_masks(
        encoder_input, output)
  
    # predictions.shape == (batch_size, seq_len, vocab_size)
    predictions, attention_weights = transformer(encoder_input, 
                                                 output,
                                                 False,
                                                 enc_padding_mask,
                                                 combined_mask,
                                                 dec_padding_mask)
    
    # select the last word from the seq_len dimension
    predictions = predictions[: ,-1:, :]  # (batch_size, 1, vocab_size)

    predicted_id = tf.argmax(predictions, axis=-1)

    # concatentate the predicted_id to the output which is given to the decoder
    # as its input.
    output = tf.concat([output, predicted_id], axis=-1)
    
    # return the result if the predicted_id is equal to the end token
    if predicted_id == end:
      break

  # output.shape (1, tokens)
  text = tokenizers.py.detokenize(output)[0] # shape: ()
  
  tokens = tokenizers.py.lookup(output)[0]
  return text, tokens, attention_weights

In [ ]:
def print_translation(sentence, tokens, ground_truth):
  print(f'{"Input:":15s}: {sentence}')
  print(f'{"Prediction":15s}: {tokens.numpy().decode("utf-8")}')
  print(f'{"Ground truth":15s}: {ground_truth}')

In [ ]:
sentence = "for every a in lista,"
ground_truth = "for a in lista:"

translated_text, translated_tokens, attention_weights = evaluate(sentence)
print_translation(sentence, translated_text, ground_truth)
# print(attention_weights)

Input:         : for every a in lista,
Prediction     : for a in lista :
Ground truth   : for a in lista:


In [ ]:
sentence = "import module json"
ground_truth = "import json"

translated_text, translated_tokens, attention_weights = evaluate(sentence)
print_translation(sentence, translated_text, ground_truth)

Input:         : import module json
Prediction     : import json
Ground truth   : import json


In [ ]:
sentence = "c is equal to a + b "
ground_truth = "c = a + b"

translated_text, translated_tokens, attention_weights = evaluate(sentence)
print_translation(sentence, translated_text, ground_truth)

Input:         : c is equal to a + b 
Prediction     : c = = ' % s ' % c
Ground truth   : c = a + b


In [ ]:
sentence = "list1 is a list containing 3 elements 1,2,3 "
ground_truth = "list1 = [1,2,3]"

translated_text, translated_tokens, attention_weights = evaluate(sentence)
print_translation(sentence, translated_text, ground_truth)

Input:         : list1 is a list containing 3 elements 1,2,3 
Prediction     : list1 = [ 2 : ] , 2 :
Ground truth   : list1 = [1,2,3]


In [ ]:
sentence = "while a is greater than b"
ground_truth = "while a>b:"

translated_text, translated_tokens, attention_weights = evaluate(sentence)
print_translation(sentence, translated_text, ground_truth)

Input:         : while a is greater than b
Prediction     : while a > b ' :
Ground truth   : while a>b:


In [ ]:
sentence= "increment num by 1"
ground_truth = "num +=1"
translated_text, translated_tokens, attention_weights = evaluate(sentence)
print_translation(sentence, translated_text, ground_truth)

Input:         : increment num by 1
Prediction     : num + = 1
Ground truth   : num +=1


In [ ]:
sentence = "num is equal to 5"
ground_truth = "num = 5"
translated_text, translated_tokens, attention_weights = evaluate(sentence)
print_translation(sentence, translated_text, ground_truth)

Input:         : num is equal to 5
Prediction     : num = 5
Ground truth   : num = 5


In [ ]:
sentence = "set num to num + 1"
ground_truth = "num = num + 1"
translated_text, translated_tokens, attention_weights = evaluate(sentence)
print_translation(sentence, translated_text, ground_truth)

Input:         : set num to num + 1
Prediction     : num + = num
Ground truth   : num = num + 1


In [ ]:
sentence = "class Person inherits from class Student"
ground_truth = "class A(B):"
translated_text, translated_tokens, attention_weights = evaluate(sentence)
print_translation(sentence, translated_text, ground_truth)

Input:         : class Person inherits from class Student
Prediction     : class persersersent ( stunders ) :
Ground truth   : class A(B):


In [ ]:
sentence = "class Person is a subclass of Student"
ground_truth = "class Person(Student):"
translated_text, translated_tokens, attention_weights = evaluate(sentence)
print_translation(sentence, translated_text, ground_truth)

Input:         : class Person is a subclass of Student
Prediction     : pers = subclass ( strudunders _ for a , medient inclass )
Ground truth   : class Person(Student):


In [ ]:
train = {}
test = {}
with open('all.anno') as json_file: 
    en = json_file.read()
    en = en.splitlines()
    new_en = []
    for line in en:
      new_en.append(line.strip())
    print(new_en)
    print(len(new_en))
with open('all.code') as json_file: 
    py = json_file.read()
    py = py.splitlines()
    new_py = []
    for line in py:
      new_py.append(line.strip())
    print(new_py)
    print(len(new_py))

['from threading import local into default name space.', 'import module warnings.', 'from django.conf import settings into default name space.', 'from django.core import signals into default name space.', 'from django.core.cache.backends.base import InvalidCacheBackendError, CacheKeyWarning and BaseCache into default name space.', 'from django.core.exceptions import ImproperlyConfigured into default name space.', 'from django.utils.deprecation import RemovedInDjango19Warning into default name space.', 'from django.utils.module_loading import import_string into default name space.', "__all__ is an list containing 6 elements: 'get_cache', 'cache', 'DEFAULT_CACHE_ALIAS', 'InvalidCacheBackendError',", "'CacheKeyWarning' and 'BaseCache'   DEFAULT_CACHE_ALIAS is a string 'default'.", 'if DEFAULT_CACHE_ALIAS is not contained in settings.CACHES,', 'raise an exception ImproperlyConfigured with string "You must define a \'%s\' cache" as argument, with \'%s\' replaced with DEFAULT_CACHE_ALIAS con

In [ ]:
train_dict = {}
train_dict['en'] = []
train_dict['py'] = []
for x ,y in zip(new_en[:15000], new_py[:15000]):
  # print(x,y)
  train_dict['en'].append(x)
  train_dict['py'].append(y)

In [ ]:
test_dict = {}
test_dict['en'] = []
test_dict['py'] = []
for x ,y in zip(new_en[15000:], new_py[15000:]):
  # print(x,y)
  test_dict['en'].append(x)
  test_dict['py'].append(y)

In [ ]:
train_df = pd.DataFrame(train_dict)
val_df = pd.DataFrame(test_dict)
print(train_df)
print(val_df)

                                                      en                                                 py
0      from threading import local into default name ...                        from threading import local
1                                import module warnings.                                    import warnings
2      from django.conf import settings into default ...                 from django . conf import settings
3      from django.core import signals into default n...                  from django . core import signals
4      from django.core.cache.backends.base import In...  from django . core . cache . backends . base i...
...                                                  ...                                                ...
14995  define the method configure_custom with 2 argu...           def configure_custom ( self , config ) :
14996  remove config dictionary element under '()' ke...                          c = config . pop ( '()' )
14997  if c doesnt have an a

In [ ]:
from nltk.translate.bleu_score import sentence_bleu

In [ ]:
sentence_bleu('class Person ( Student ) :'.split(), 'class persersersent ( stunders ) :'.split())

/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


0.8408964152537145

In [ ]:
predictions = []
for i in range(len(test_dict['py'])):
  sentence=test_dict['en'][i]
  ground_truth = test_dict['py'][i]
  translated_text, translated_tokens, attention_weights = evaluate(sentence)
  predictions.append(translated_text.numpy().decode("utf-8"))
predictions

["kwargs = dict ( ( config , k ) for k , v in valid _ id if not in valid _ state [ ' cache - caent ' ] )",
 'result = c ( * * kwargs )',
 'if props :',
 'for name , value in props ( dictionary ) :',
 'result = setattr ( value , value )',
 'return result',
 'def as _ tuple ( self , value ) :',
 'if isinstance ( val , list ) :',
 'value = tuple ( value )',
 'return value',
 'class dictconfigurator ( basecon , basecontrigrator ) :',
 'def configure ( self ) :',
 'config = self . config',
 "if ' version ' not in config :",
 'raise valueerror ( " dict doesn \' t specify version " )',
 "if config . get ( ' version ' ) ! = 1 :",
 'raise valueerror ( " unsupported version \' % s \' did not re \' didd \' " % d instead . _ _ class _ _ . keys ( key ) )',
 "module = config . get ( ' + ' , ' . join ( config ) )",
 'notal = false + { }',
 'logging . _ acquire ( )',
 'try :',
 'if i + :',
 "handlers _ dict = set ( config . get ( ' handlers ' , { } ) )",
 'if sys . version [ : 2 ] = = range ( 7 , 3 ) 

In [ ]:
bleu_scores = []
for i in range(len(predictions)):
  bleu_scores.append(sentence_bleu(test_dict['py'][i].split(), predictions[i].split()))
bleu_scores

/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


[0.7340408068222942,
 0.5721248424548514,
 0.7598356856515925,
 0.7952707287670506,
 0.7825422900366437,
 0,
 0.8408964152537145,
 0.6548907866815301,
 0.8408964152537145,
 0,
 0.7825422900366437,
 0.8408964152537145,
 0.7952707287670506,
 0.6905911470987942,
 0.44116293593227063,
 0.4428500142691474,
 0.2996709045159163,
 0.8010548969451144,
 0.43173061492439624,
 0.7648825826302762,
 0.8408964152537145,
 0.8408964152537145,
 0.816496580927726,
 0.4484416535458776,
 0.668740304976422,
 0.7598356856515925,
 0.23356898886410005,
 0.8408964152537145,
 0.8408964152537145,
 0.8633400213704505,
 0.8408964152537145,
 0.7397701184050692,
 0.7598356856515925,
 0.8408964152537145,
 0.6511126026643229,
 0.23249999518652367,
 0.8545740127924681,
 0.668740304976422,
 0.8408964152537145,
 0.5756816179786388,
 0.6511126026643229,
 0.23249999518652367,
 0.8739351325046805,
 0.7598356856515925,
 0.8408964152537145,
 0.563669723784074,
 0.6511126026643229,
 0.2676032275663791,
 0.8408964152537145,
 0.7

In [ ]:
sum(bleu_scores)/len(bleu_scores)

0.6429849264443741